In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score

# Instala la librería ucimlrepo y otras dependencias
!pip install ucimlrepo
!pip install matplotlib seaborn

from ucimlrepo import fetch_ucirepo

### Cargar los datos desde el repositorio UCI

In [ ]:
# Función para cargar los datos del conjunto de datos UCI con ID 565
def load_data():
    bmtch = fetch_ucirepo(id=565)
    X = bmtch.data.features
    y = bmtch.data.targets
    data = pd.concat([X, y], axis=1)
    return data, bmtch

### Exploración inicial de los datos

In [ ]:
# Explora los datos cargados
def explore_data(data, metadata):
    print(data.head().T)
    print(data.describe())
    print(data.info())
    print("Metadata:", metadata)

### Imputación de valores faltantes en las columnas

In [ ]:
# Imputación de valores
def imputar_valores(data):
    columnas_moda = ['RecipientABO', 'RecipientRh', 'ABOmatch', 'CMVstatus', 'DonorCMV', 'RecipientCMV', 'Antigen', 'Allele', 'extcGvHD']
    columnas_mediana = ['CD3dCD34', 'CD3dkgx10d8', 'Rbodymass']
    
    for col in columnas_moda:
        data[col].fillna(data[col].mode()[0], inplace=True)
    
    for col in columnas_mediana:
        data[col].fillna(data[col].median(), inplace=True)
    
    return data

### Detección de valores erróneos

In [ ]:
# Detección de valores erróneos según reglas predefinidas
def detect_errors(df, column, rule):
    return df[~df[column].apply(rule)]

### Eliminación de valores erróneos e imputación de la columna específica

In [ ]:
# Función para eliminar valores erróneos y realizar imputación
def eliminar_e_imputar(data, error_rules):
    indices_a_eliminar = []
    for col, rule in error_rules.items():
        if col != 'time_to_aGvHD_III_IV':
            errores = detect_errors(data, col, rule)
            indices_a_eliminar.extend(errores.index.tolist())
    
    data.drop(index=set(indices_a_eliminar), inplace=True)
    mediana_aGvHD = data.loc[data['time_to_aGvHD_III_IV'] != 1000000, 'time_to_aGvHD_III_IV'].median()
    data['time_to_aGvHD_III_IV'].replace(1000000, mediana_aGvHD, inplace=True)
    return data

### Graficar histogramas de las variables numéricas seleccionadas

In [ ]:
# Graficar histogramas para las variables seleccionadas
def plot_histograms(data, selected_columns):
    plt.figure(figsize=(12, 10))
    for i, col in enumerate(selected_columns, 1):
        plt.subplot(5, 2, i)
        plt.hist(data[col], bins=30, color='blue', edgecolor='black')
        plt.title(f'Histograma de {col}')
        plt.xlabel(col)
        plt.ylabel('Frecuencia')
    plt.tight_layout()
    plt.show()

### Graficar Boxplots de las variables numéricas seleccionadas

In [ ]:
# Graficar Boxplots para las variables seleccionadas
def plot_boxplots(data, selected_columns):
    plt.figure(figsize=(15, 15))
    n_cols = 5
    n_rows = (len(selected_columns) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 15))
    
    for i, col in enumerate(selected_columns):
        row = i // n_cols
        col_num = i % n_cols
        sns.boxplot(data=data, y=col, ax=axes[row, col_num], color="skyblue")
        axes[row, col_num].set_title(f'Boxplot de {col}')
    
    for j in range(i + 1, n_rows * n_cols):
        fig.delaxes(axes.flatten()[j])
    
    plt.tight_layout()
    plt.show()

### Graficar la matriz de correlación entre variables

In [ ]:
# Graficar la matriz de correlación
def plot_correlation_matrix(data, selected_columns):
    plt.figure(figsize=(12, 8))
    sns.heatmap(data[selected_columns].corr(), annot=True, fmt=".2f", cmap='coolwarm')
    plt.show()

### Ejecución del pipeline de limpieza y visualización de datos

In [ ]:
# Main function
def main():
    data, bmtch = load_data()
    explore_data(data, bmtch.metadata)

    data = imputar_valores(data)

    # Reglas de error completas
    error_rules = {
        'Recipientgender': lambda x: x in [0, 1],  # 0: Femenino, 1: Masculino
        'Stemcellsource': lambda x: x in [0, 1],  # 0: Sangre periférica, 1: Médula ósea
        'Donorage': lambda x: (0 <= x <= 120),  # Edad del donante fuera del rango
        'Donorage35': lambda x: x in [0, 1],  # Edad del donante fuera del rango
        'IIIV': lambda x: x in [0, 1],  # Clasificación del trasplante entre I y IV
        'Gendermatch': lambda x: x in [0, 1],  # 0: No compatible, 1: Compatible
        'DonorABO': lambda x: x in [-1, 0, 1, 2],  # -1: B, 0: O, 1: A, 2: AB
        'RecipientABO': lambda x: x in [-1, 0, 1, 2],  # -1: B, 0: O, 1: A, 2: AB
        'RecipientRh': lambda x: x in [0, 1],  # 0: Rh-, 1: Rh+
        'ABOmatch': lambda x: x in [0, 1],  # 0: Mismatched, 1: Matched
        'CMVstatus': lambda x: x in [0, 1, 2, 3],  # Presencia/ausencia de CMV y otros estados
        'DonorCMV': lambda x: x in [0, 1],  # 0: Negativo, 1: Positivo
        'RecipientCMV': lambda x: x in [0, 1],  # 0: Negativo, 1: Positivo
        'Disease': lambda x: x in ['ALL', 'AML', 'chronic', 'nonmalignant', 'lymphoma'],  # Tipos de enfermedad
        'Riskgroup': lambda x: x in [0, 1],  # 0: Bajo, 1: Alto
        'Txpostrelapse': lambda x: x in [0, 1],  # 0: No, 1: Sí (trasplante tras recaída)
        'Diseasegroup': lambda x: x in [0, 1],  # 0: No maligna, 1: Maligna
        'HLAmatch': lambda x: x in [0, 1, 2, 3],  # Compatibilidad HLA 7/10 a 10/10
        'HLAmismatch': lambda x: x in [0, 1],  # 0: Compatible, 1: No compatible
        'Antigen': lambda x: x in [-1, 0, 1, 2],  # Diferencia de antígenos entre donante y receptor
        'Allele': lambda x: x in [-1, 0, 1, 2, 3],  # Diferencia de alelos entre donante y receptor
        'HLAgrI': lambda x: (0 <= x <= 5),  # Grado de diferencia HLA (fuera de rango)
        'Recipientage': lambda x: (0 <= x <= 120),  # Edad del receptor fuera de rango
        'Recipientage10': lambda x: x in [0, 1],  # 0: >= 10 años, 1: < 10 años
        'Recipientageint': lambda x: x in [0, 1, 2],  # Edad del receptor discretizada en intervalos
        'Relapse': lambda x: x in [0, 1],  # 0: No, 1: Sí (recaída)
        'aGvHDIIIIV': lambda x: x in [0, 1, 2, 3],  # Grado de GvHD aguda entre 0 y IV
        'extcGvHD': lambda x: x in [0, 1],  # 0: No, 1: Sí (GvHD crónica extensa)
        'CD34kgx10d6': lambda x: (0 <= x <= 60),  # Dosis de CD34+ fuera de rango
        'CD3dCD34': lambda x: (0 <= x <= 100),  # Relación CD3/CD34 fuera de rango
        'CD3dkgx10d8': lambda x: (0 <= x <= 21),  # Dosis de CD3+ fuera de rango
        'Rbodymass': lambda x: (6 <= x <= 104),  # Masa corporal del receptor fuera de rango
        'ANCrecovery': lambda x: (7 <= x <= 30),  # Tiempo de recuperación de neutrófilos fuera de rango
        'PLTrecovery': lambda x: (0 <= x <= 365),  # Tiempo de recuperación de plaquetas fuera de rango
        'time_to_aGvHD_III_IV': lambda x: (0 <= x <= 365),  # Tiempo en días hasta GvHD aguda
        'survival_time': lambda x: (0 <= x <= 4000),  # Tiempo de supervivencia fuera de rango
        'survival_status': lambda x: x in [0, 1],  # 0: Vivo, 1: Fallecido
    }

    data = eliminar_e_imputar(data, error_rules)

    selected_columns = ['Donorage', 'Recipientage', 'Rbodymass', 'CD34kgx10d6', 'CD3dkgx10d8',
                        'CD3dCD34', 'ANCrecovery', 'PLTrecovery', 'time_to_aGvHD_III_IV', 'survival_time']

    plot_histograms(data, selected_columns)
    plot_boxplots(data, selected_columns)
    plot_correlation_matrix(data, selected_columns)

main()